# Advanced 3D Reconstruction Visualization for Interior Reimaginer

This notebook demonstrates advanced 3D visualization techniques for the Interior Reimaginer project, highlighting superior alternatives to standard Plotly visualizations. We'll showcase GAN and diffusion-based depth enhancement methods for improved 3D reconstruction quality.

## Setup

First, let's clone the repository and install the required dependencies:

In [ ]:
# Clone the repository
!git clone https://github.com/your-username/CV_Project.git

%cd CV_Project

In [ ]:
# Install dependencies
!pip install -r interior_reimaginer/requirements.txt

# Additional packages for advanced visualization
!pip install open3d matplotlib plotly pyrender trimesh pyglet==1.5.27 PyOpenGL PyOpenGL-accelerate ipywidgets ipyvolume k3d pythreejs

## Fix Import Issues

Let's modify the `__init__.py` files to avoid import errors:

In [ ]:
# Create a temporary fix for the models/__init__.py file to avoid UI import
%%writefile interior_reimaginer/models/__init__.py
# models/__init__.py
from .image_processor import ImageProcessor, ProcessedImage, RoomSegmentClass
from .interior_reimaginer import InteriorReimaginer
from .design_styles import DesignStyle, load_design_styles

__all__ = [
    'ImageProcessor', 
    'ProcessedImage', 
    'RoomSegmentClass', 
    'InteriorReimaginer', 
    'DesignStyle', 
    'load_design_styles'
]

## Import Required Modules

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import open3d as o3d
from PIL import Image
import cv2
import time
import io
import base64
from IPython.display import HTML, display
import ipyvolume as ipv
import plotly.graph_objects as go
from IPython.display import IFrame

# Make sure Python can find our modules
sys.path.append('/content/CV_Project')

# Import our project modules directly to avoid potential issues
try:
    from interior_reimaginer.models.interior_reimaginer import InteriorReimaginer
    from interior_reimaginer.models.reconstruction_3d import DepthReconstructor
    print("Successfully imported project modules")
except Exception as e:
    print(f"Error importing modules: {e}")
    print("Attempting alternative imports...")
    # Try direct imports as a fallback
    sys.path.insert(0, '/content/CV_Project/interior_reimaginer')
    from models.reconstruction_3d import DepthReconstructor
    from models.interior_reimaginer import InteriorReimaginer

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Initialize models
print("Initializing models...")
try:
    reimaginer = InteriorReimaginer(device=device)
    depth_reconstructor = DepthReconstructor()
    print("Models initialized successfully!")
except Exception as e:
    print(f"Error initializing models: {e}")
    print("Initializing only DepthReconstructor as fallback...")
    depth_reconstructor = DepthReconstructor()

## Upload or Create Test Image

You can either upload your own interior image or use a test image:

In [ ]:
# Option 1: Upload an image
try:
    from google.colab import files
    uploaded = files.upload()  # This will prompt you to upload an image
    
    if uploaded:
        image_name = list(uploaded.keys())[0]
        image = Image.open(io.BytesIO(uploaded[image_name]))
    else:  # If no upload, create a simple test image
        raise ValueError("No image uploaded")
except Exception as e:
    print(f"Error with image upload: {e}")
    print("Creating a test image...")
    # Create a simple test image with a gradient
    width, height = 800, 600
    img_array = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(height):
        for j in range(width):
            img_array[i, j] = [int(255*i/height), int(255*j/width), 100]
    image = Image.fromarray(img_array)

# Display the image
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.title("Image for Processing")
plt.axis('off')
plt.show()

## Process the Image

We'll process the image to extract depth information:

In [ ]:
print("Processing the image...")
depth_map = None
processed = None

try:
    # Try to use the reimaginer's image_processor if available
    processed = reimaginer.image_processor.process_image(image)
    if processed and processed.depth_map is not None:
        depth_map = processed.depth_map
        print("Image processing complete with reimaginer!")
    else:
        raise ValueError("No depth map generated by reimaginer")
except Exception as e:
    print(f"Error processing with reimaginer: {e}")
    print("Creating a synthetic depth map for visualization...")
    
    # Create a simple synthetic depth map if processing fails
    img_array = np.array(image)
    height, width = img_array.shape[:2]
    
    # Create a gradient depth map
    x = np.linspace(0, 1, width)
    y = np.linspace(0, 1, height)
    xx, yy = np.meshgrid(x, y)
    # Make depth a function of distance from center
    center_x, center_y = width / 2, height / 2
    xx_centered = xx * width - center_x
    yy_centered = yy * height - center_y
    depth_map = np.sqrt(xx_centered**2 + yy_centered**2)
    
    # Normalize to 0-255 range
    depth_map = 255 * (1 - depth_map / depth_map.max())
    depth_map = depth_map.astype(np.uint8)

# Display the depth map
if depth_map is not None:
    plt.figure(figsize=(12, 8))
    plt.imshow(depth_map, cmap='inferno')
    plt.title("Depth Map")
    plt.colorbar(label="Depth")
    plt.axis('off')
    plt.show()
else:
    print("Depth map generation failed.")

## Advanced 3D Visualization Methods

Let's explore different advanced visualization techniques beyond standard Plotly:

### 1. Enhanced Depth Map Visualization

In [ ]:
# Apply high-quality render with improved colormaps
try:
    enhanced_depth_viz = depth_reconstructor.render_depth_map(
        depth_map, 
        colormap=cv2.COLORMAP_TURBO,  # Try different colormaps: COLORMAP_TURBO, COLORMAP_JET, COLORMAP_VIRIDIS
        width=1000, 
        height=800
    )
    
    plt.figure(figsize=(16, 10))
    plt.imshow(enhanced_depth_viz)
    plt.title("Enhanced Depth Map Visualization")
    plt.axis('off')
    plt.show()
except Exception as e:
    print(f"Error rendering enhanced depth map: {e}")
    # Fallback to basic visualization
    plt.figure(figsize=(16, 10))
    plt.imshow(depth_map, cmap='turbo')
    plt.title("Basic Depth Map Visualization")
    plt.axis('off')
    plt.colorbar(label="Depth")
    plt.show()

### 2. GAN-Based Enhanced 3D Reconstruction

Use GAN-based depth enhancement to improve the quality of the 3D reconstruction:

In [ ]:
print("Creating GAN-enhanced 3D reconstruction...")
try:
    gan_render_img, gan_pcd = depth_reconstructor.enhanced_reconstruction(
        depth_map=depth_map,
        image=image,
        width=1000,
        height=800,
        use_gan=True,  # Enable GAN enhancement
        solid_rendering=True  # Use solid rendering for more realistic visualization
    )
    
    plt.figure(figsize=(16, 10))
    plt.imshow(gan_render_img)
    plt.title("GAN-Enhanced 3D Reconstruction")
    plt.axis('off')
    plt.show()
except Exception as e:
    print(f"Error with GAN-enhanced reconstruction: {e}")

### 3. Diffusion-Based Enhanced 3D Reconstruction

Use diffusion-based depth enhancement for a different approach to improving 3D reconstruction:

In [ ]:
print("Creating diffusion-enhanced 3D reconstruction...")
try:
    diffusion_render_img, diffusion_pcd = depth_reconstructor.diffusion_reconstruction(
        depth_map=depth_map,
        image=image,
        width=1000,
        height=800,
        solid_rendering=True  # Use solid rendering for more realistic visualization
    )
    
    plt.figure(figsize=(16, 10))
    plt.imshow(diffusion_render_img)
    plt.title("Diffusion-Enhanced 3D Reconstruction")
    plt.axis('off')
    plt.show()
except Exception as e:
    print(f"Error with diffusion-enhanced reconstruction: {e}")

### 4. Local Region Models (LRM) Reconstruction

Use LRM for more detailed local geometry processing:

In [ ]:
print("Creating LRM 3D reconstruction...")
try:
    lrm_render_img, lrm_pcd = depth_reconstructor.lrm_reconstruction(
        depth_map=depth_map,
        image=image,
        width=1000,
        height=800,
        downsample_factor=2,
        patch_size=32,
        overlap=8,
        solid_rendering=True
    )
    
    plt.figure(figsize=(16, 10))
    plt.imshow(lrm_render_img)
    plt.title("LRM-Enhanced 3D Reconstruction")
    plt.axis('off')
    plt.show()
except Exception as e:
    print(f"Error with LRM reconstruction: {e}")

## Interactive 3D Visualization with HTML Export

In [ ]:
def create_interactive_pointcloud_visualization(pcd, width=900, height=600):
    """Create an interactive 3D visualization using Plotly"""
    # Convert Open3D point cloud to numpy arrays
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    
    # Convert RGB to hex for Plotly
    hex_colors = [f'rgb({int(r*255)},{int(g*255)},{int(b*255)})' for r, g, b in colors]
    
    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 2],  # Use z as y for better orientation
        z=-points[:, 1],  # Negative y for correct orientation
        mode='markers',
        marker=dict(
            size=2,
            color=hex_colors,
            opacity=0.8
        )
    )])
    
    # Set up the layout
    fig.update_layout(
        width=width,
        height=height,
        scene=dict(aspectmode='data'),
        margin=dict(l=0, r=0, b=0, t=30),
        title="Interactive 3D Reconstruction"
    )
    
    return fig

# Initialize default point cloud if needed
if 'gan_pcd' not in locals() and 'diffusion_pcd' not in locals() and 'lrm_pcd' not in locals():
    print("Creating a simple point cloud for visualization...")
    # Create a simple point cloud
    pcd = o3d.geometry.PointCloud()
    points = []
    colors = []
    for i in range(1000):
        points.append([(np.random.rand()-0.5)*2, (np.random.rand()-0.5)*2, (np.random.rand()-0.5)*2])
        colors.append([np.random.rand(), np.random.rand(), np.random.rand()])
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    best_pcd = pcd
else:
    # Use the best available point cloud based on point count
    if 'lrm_pcd' in locals() and len(lrm_pcd.points) > 0:
        best_pcd = lrm_pcd
    elif 'diffusion_pcd' in locals() and len(diffusion_pcd.points) > 0:
        best_pcd = diffusion_pcd
    elif 'gan_pcd' in locals() and len(gan_pcd.points) > 0:
        best_pcd = gan_pcd
    else:
        # Create a simple point cloud
        pcd = o3d.geometry.PointCloud()
        points = []
        colors = []
        for i in range(1000):
            points.append([(np.random.rand()-0.5)*2, (np.random.rand()-0.5)*2, (np.random.rand()-0.5)*2])
            colors.append([np.random.rand(), np.random.rand(), np.random.rand()])
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        best_pcd = pcd

# Create and display interactive visualization
try:
    interactive_fig = create_interactive_pointcloud_visualization(best_pcd)
    interactive_fig.show()
except Exception as e:
    print(f"Error creating interactive visualization: {e}")

## Mesh Visualization

Create and visualize a 3D mesh from the point cloud:

In [ ]:
print("Creating mesh from point cloud...")
try:
    if len(best_pcd.points) > 100:
        # Smooth the point cloud for better mesh generation
        best_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        best_pcd.orient_normals_towards_camera_location()
        
        # Create mesh
        mesh = depth_reconstructor.pointcloud_to_mesh(best_pcd)
        print(f"Mesh created with {len(mesh.triangles)} triangles")
        
        # Render mesh
        mesh_image = depth_reconstructor.render_mesh_image(mesh, width=1000, height=800)
        
        plt.figure(figsize=(16, 10))
        plt.imshow(mesh_image)
        plt.title("3D Mesh Reconstruction")
        plt.axis('off')
        plt.show()
        
        # Save mesh to OBJ file
        timestamp = int(time.time())
        mesh_path = depth_reconstructor.save_mesh(mesh, f"interior_mesh_{timestamp}")
        print(f"Mesh saved to: {mesh_path}")
        
        # Download the mesh file
        from google.colab import files
        files.download(mesh_path)
    else:
        print("Not enough points in point cloud for mesh generation")
except Exception as e:
    print(f"Mesh creation failed: {e}")

## Conclusion

In this notebook, we've demonstrated several advanced 3D visualization techniques for interior reconstruction:

1. Enhanced depth map rendering with improved colormaps
2. GAN-based depth enhancement for better 3D reconstruction
3. Diffusion-based methods for filling in missing depth information
4. Local Region Models (LRM) for detailed geometry processing
5. Interactive 3D visualizations and mesh creation

These techniques provide significant improvements over standard Plotly visualizations, offering better detail, more realistic rendering, and enhanced interactivity for exploring 3D reconstructions.